## CSE 4705: Assignment 02 - Arad to Bucharest - BFS, DFS, UCS, GBFS, A* 


## Problem 1
[100] Write a routine that solves the problem of finds a travel path of cities from from Arad to Bucharest in Romania, as discussed in class. Do this using each of the following approaches (points shown in brackets):

1. [15] Breadth First Search (BFS)
2. [10] Depth First Search (DFS)
3. [25] Uniform Cost Search (UCS)
4. [25] Greedy Best First Search (GBFS)
5. [25] A*


You will use the map from Lecture 03 - Informed Search which shows the major cities in Romania and the distances between them for those cities that are directly connected.  Also, you will use the straight-line-distances shown in the adjacent table for your heuristic function, $h(n)$ for GBFS and A*.  A screenshot of the relevant slide is given below.  Data structures that store this information, romania_map and sld_to_bucharest, have been provided so you can access and apply this data in your algorithm implementations.  Details of these data structures are given below.

<img src="images/romania_cities.png" width="800" height="600">

### Output for Each Routine

Each of your routines should return an output or set of outputs that clearly indicates the following:

1. The sequence of cities from Arad to Bucharest.  (Make sure the cities, Arad and Bucharest are explicitly listed as the first and last cities in your output.)  One suggestion is to return this output in the form of a list.
2. Cost to travel to each city from its predecessor.  
3. Total cost for the path.  

In the case of A* and Uniform Cost Search, your routines should return the *cheapest path*.  However, that will not necessarily be the case for BFS, DFS, or GBFS.  (Why not?)

### Romania Graph

You will use the data structure stored in the romania_map, assigned below to implement the search across the various cities to find a path from Arad to Bucharest.

Some details about romania_map:
- A dictionary of dictionaries 
- The outer dictionary is as follows: each key is a city and the value for that city is a nested dictionary of cities to which the said city is directly connected.
- The nested dictionary contains the cities to which the parent key is directly connected (keys) and the corresponding distances from the parent city to those respective cities (values).
- For example, for the city Oradea, we have a key in the outer dictionary (Oradea), and the associated value is a dictionary containing the Zerind and Sibiu as keys, where for each of these the values are the distances from Oradea to these respective cities.



In [5]:

romania_map = {
    'Oradea':{'Zerind':71, 'Sibiu':151},
    'Zerind':{'Oradea':71, 'Arad':75},
    'Arad':{'Zerind':75, 'Sibiu':140, 'Timisoara':118},
    'Timisoara':{'Arad':118, 'Lugoj':111},
    'Lugoj':{'Timisoara':111, 'Mehadia':70},
    'Mehadia':{'Lugoj':70, 'Dobreta':75},
    'Dobreta':{'Mehadia':75, 'Craiova':120},
    'Sibiu':{'Oradea':151, 'Fagaras':99, 'Rimnicu Vilcea':80, 'Arad':140},
    'Rimnicu Vilcea':{'Sibiu':80, 'Pitesti':97, 'Craiova':146},
    'Craiova':{'Rimnicu Vilcea':146, 'Pitesti':138, 'Dobreta':120},
    'Fagaras':{'Sibiu':99, 'Bucharest':211},
    'Pitesti':{'Rimnicu Vilcea':97, 'Bucharest':101, 'Craiova':138},
    'Neamt':{'Iasi':87},
    'Giurgiu':{'Bucharest':90},
    'Bucharest':{'Pitesti':101, 'Fagaras':211, 'Urziceni':85, 'Giurgiu':90},
    'Iasi':{'Neamt':87, 'Vaslui':92},
    'Urziceni':{'Bucharest':85, 'Vaslui':142, 'Hirsova':98},
    'Vaslui':{'Iasi':92, 'Urziceni':142},
    'Hirsova':{'Urziceni':98, 'Eforie':86},
    'Eforie':{'Hirsova':86}
}

### Heuristic Function Data - Straight-Line Distances to Bucharest

You will use the dictionary below as your resource for retrieving straight-line distance data for implementing the GBFS and A* algorithms.

In [6]:
sld_to_Bucharest = {'Arad':366,
                    'Bucharest':0,
                    'Craiova':160,
                    'Dobreta':242,
                    'Eforie':161,
                    'Fagaras':176,
                    'Giurgiu':77,
                    'Hirsova':151,
                    'Iasi':226,
                    'Lugoj':244,
                    'Mehadia':241,
                    'Neamt':234,
                    'Oradea':380,
                    'Pitesti':100,
                    'Rimnicu Vilcea':193,
                    'Sibiu':253,
                    'Timisoara':329,
                    'Urziceni':80,
                    'Vaslui':199,
                    'Zerind':374
                   }

### 1. BFS Implementation

Provide your implementation of the BFS Search below.

In [8]:
from collections import deque
# import queue

class Node:
  def __init__(self, name, predecessor, cost):
    self.name = name
    self.predecessor = predecessor
    self.cost = cost

class Queue:
  def __init__(self):
    self.container = deque()
  
  def put(self, data):
    self.container.append(data)

  def get(self):
    return self.container.popleft()
  
  def empty(self):
    return len(self.container) == 0

class Stack:
  def __init__(self):
    self.container = deque()

  def put(self, data):
    self.container.append(data)

  def get(self):
    return self.container.pop()

  def empty(self):
    return len(self.container) == 0
  
def path(node):
  if node.predecessor == None:
    return [node.name, node.cost]
  else:
    return path(node.predecessor) + [node.name, node.cost]
  
def report_path(path):
  print("Path: ", end='')
  for i in range(0, len(path), 2):
    print(path[i], '(' + str(path[i+1]) + ')', end='')
    if i < len(path) - 2:
      print(" -> ", end='')
  print("\nCost: ", path[-1])

In [9]:
def searchGeneric(graph, start, goal, search_type, debug=False):  
  if search_type == 'bfs':
    frontier = Queue()
  elif search_type == 'dfs':
    frontier = Stack()
  else:
    raise Exception('Invalid search type')
  
  visited_nodes = set()

  # Create a node for the start node
  start_node = Node(start, None, 0)

  visited_nodes.add(start_node.name)
  frontier.put(start_node)

  while (not frontier.empty()):
    current_node = frontier.get()

    # For all the neighbor nodes of the current node
    for neighbor in graph[current_node.name]:
      if neighbor in visited_nodes:
        continue
      neighbor_node = Node(neighbor, current_node, current_node.cost + graph[current_node.name][neighbor])
      if (debug): print('Visiting node: ', neighbor_node.name)
      if neighbor_node.name == goal:
        return path(neighbor_node)
      visited_nodes.add(neighbor_node.name)
      frontier.put(neighbor_node)

  return None

In [14]:
def bfs(graph, start_node, goal_node):
  return searchGeneric(graph, start_node, goal_node, 'bfs', debug=False)

In [15]:
res = bfs(romania_map, 'Arad', 'Bucharest')
report_path(res)

Path: Arad (0) -> Sibiu (140) -> Fagaras (239) -> Bucharest (450)
Cost:  450


### 2.  DFS Implementation

Provide your implementation of the DFS Search below.

In [16]:
def dfs(graph, start_node, goal_node):
  return searchGeneric(graph, start_node, goal_node, 'dfs')

In [18]:
res = dfs(romania_map, 'Arad', 'Bucharest')
report_path(res)

Path: Arad (0) -> Timisoara (118) -> Lugoj (229) -> Mehadia (299) -> Dobreta (374) -> Craiova (494) -> Pitesti (632) -> Bucharest (733)
Cost:  733


### 3. UCS Implementation

Provide your implementation of the UCS Search below.

In [19]:
import heapq

def path(node):
  if node.predecessor == None:
    return [node.name, node.cost]
  else:
    return path(node.predecessor) + [node.name, node.cost]
  
def report_path(path):
  print("Path: ", end='')
  for i in range(0, len(path), 2):
    print(path[i], '(' + str(path[i+1]) + ')', end='')
    if i < len(path) - 2:
      print(" -> ", end='')
  print("\nCost: ", path[-1])

class PriorityQueue:
  def __init__(self):
    self.container = []
    self.set = set()

  def getPriority(self, dataName):
    for (priority, d) in self.container:
      if d.name == dataName:
        return priority
    return None
  
  def getWithName(self, dataName):
    for (priority, d) in self.container:
      if d.name == dataName:
        return d
    return None

  # This function also handles duplicates with different priorities
  # Keeps the one with the lowest priority
  def put(self, data, priority):
    if not data.name in self.set:
      self.set.add(data.name)
      heapq.heappush(self.container, (priority, data))
    else:
      if priority < self.getPriority(data.name):
        oldData = self.getWithName(data.name)
        self.container.remove((self.getPriority(data.name), oldData))
        heapq.heappush(self.container, (priority, data))

  def get(self):
    return heapq.heappop(self.container)[-1]
  
  def empty(self):
    return len(self.container) == 0
  

In [20]:
def searchPriority(graph, start, goal, search_type, debug=False):
  if search_type == 'ucs':
    frontier = PriorityQueue()
  elif search_type == 'gbfs':
    frontier = PriorityQueue()
  elif search_type == 'a_star':
    frontier = PriorityQueue()
  else:
    raise Exception('Invalid search type')
  
  explored_nodes = set()

  # Create a node for the start node
  start_node = Node(start, None, 0)

  frontier.put(start_node, start_node.cost)

  while (not frontier.empty()):
    current_node = frontier.get()
    explored_nodes.add(current_node.name)
    if (current_node.name == goal):
      return path(current_node)

    # For all the neighbor nodes of the current node
    for neighbor in graph[current_node.name]:
      if neighbor in explored_nodes:
        continue
      neighbor_node = Node(neighbor, current_node, current_node.cost + graph[current_node.name][neighbor])
      if (debug): print('Visiting node: ', neighbor_node.name)

      # The put function for priority queue handles duplicate nodes with the same name
      if search_type == 'ucs':
        # f(n) = g(n)
        frontier.put(neighbor_node, neighbor_node.cost)
      elif search_type == 'gbfs':
        # f(n) = h(n)
        frontier.put(neighbor_node, sld_to_Bucharest[neighbor_node.name])
      elif search_type == 'a_star':
        # f(n) = g(n) + h(n)
        frontier.put(neighbor_node, neighbor_node.cost + sld_to_Bucharest[neighbor_node.name])

  return None

In [21]:
def ucs(graph, start_node, goal_node):
  return searchPriority(graph, start_node, goal_node, 'ucs')

In [23]:
res = ucs(romania_map, 'Arad', 'Bucharest')
report_path(res)

Path: Arad (0) -> Sibiu (140) -> Rimnicu Vilcea (220) -> Pitesti (317) -> Bucharest (418)
Cost:  418


### 4. GBFS Implementation

Provide your implementation of the GBFS Search below.

In [24]:
def gbfs(graph, start_node, goal_node):
  return searchPriority(graph, start_node, goal_node, 'gbfs')

In [25]:
res = gbfs(romania_map, 'Arad', 'Bucharest')
report_path(res)

Path: Arad (0) -> Sibiu (140) -> Fagaras (239) -> Bucharest (450)
Cost:  450


### 5. A* Implementation

Provide your implementation of the A* Algorithm below.

In [26]:
def a_star(graph, start_node, goal_node):
  return searchPriority(graph, start_node, goal_node, 'a_star')

In [28]:
res = a_star(romania_map, 'Arad', 'Bucharest')
report_path(res)

Path: Arad (0) -> Sibiu (140) -> Rimnicu Vilcea (220) -> Pitesti (317) -> Bucharest (418)
Cost:  418
